In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

219062272/219055592 [==============================] - 25s 0us/step


In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 386 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 56s - loss: 0.8423 - auc: 0.8328 - accuracy: 0.7538 - val_loss: 0.2456 - val_auc: 0.9863 - val_accuracy: 0.8724
Epoch 2/30
40/40 - 29s - loss: 0.1120 - auc: 0.9938 - accuracy: 0.9622 - val_loss: 0.0514 - val_auc: 0.9997 - val_accuracy: 0.9948
Epoch 3/30
40/40 - 28s - loss: 0.0550 - auc: 0.9995 - accuracy: 0.9937 - val_loss: 0.0414 - val_auc: 1.0000 - val_accuracy: 0.9948
Epoch 4/30
40/40 - 28s - loss: 0.0360 - auc: 1.0000 - accuracy: 0.9968 - val_loss: 0.0213 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 5/30
40/40 - 28s - loss: 0.0265 - auc: 1.0000 - accuracy: 0.9984 - val_loss: 0.0280 - val_auc: 1.0000 - val_accuracy: 0.9974
Epoch 6/30
40/40 - 28s - loss: 0.0195 - auc: 1.0000 - accuracy: 0.9992 - val_loss: 0.0153 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 7/30
40/40 - 28s - loss: 0.0163 - auc: 1.0000 - accuracy: 0.9996 - val_loss: 0.0123 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 8/30
40/40 - 27s - loss: 0.0132 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

318
68


In [8]:
prediction

array([[9.97799337e-01],
       [1.00000000e+00],
       [9.47749436e-01],
       [9.27482903e-01],
       [9.98317480e-01],
       [1.00000000e+00],
       [1.37065817e-02],
       [9.25154090e-01],
       [9.94261622e-01],
       [6.91482008e-01],
       [6.90052211e-01],
       [9.90314305e-01],
       [1.00000000e+00],
       [9.96689439e-01],
       [9.91766393e-01],
       [1.29362158e-02],
       [9.45201099e-01],
       [9.96667564e-01],
       [9.99913692e-01],
       [9.99482393e-01],
       [3.42842401e-03],
       [9.48334336e-02],
       [8.82942379e-01],
       [9.30289268e-01],
       [9.96263921e-01],
       [1.00000000e+00],
       [2.73933876e-02],
       [9.51309800e-01],
       [9.67350364e-01],
       [9.99456704e-01],
       [9.69227850e-01],
       [9.92739141e-01],
       [1.00000000e+00],
       [9.68067765e-01],
       [9.69494998e-01],
       [9.95255411e-01],
       [9.98148322e-01],
       [9.98974204e-01],
       [9.95741487e-01],
       [8.17351758e-01],


In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

7/7 [==============================] - 3s 479ms/step - loss: 0.0412 - auc: 1.0000 - accuracy: 0.9948


[0.04122127965092659, 1.0, 0.9948186278343201]